## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

### 使用 LASSO 以 diabetes 資料集進行訓練，找出最佳 α 值

In [1]:
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [43]:
# 載入糖尿病資料集
diabetes = datasets.load_diabetes()
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.25, random_state=42)

# 建立 LASSO 線性回歸模型，未指定α，採用預設值 (1.0)
lasso = linear_model.Lasso()

# 查看在預設α情況下，MSE 為何？
lasso.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

3433.1524588051197


In [44]:
# 設定要訓練的超參數組合
alpha = [0.00000000001, 0.001, 0.1, 0.25, 0.5, 1.0, 1.5, 2.0]
param_grid = dict(alpha=alpha)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(lasso, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 8 種參數組合，總共要 train 24 次模型

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    2.8s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [45]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -3184.306576 using {'alpha': 0.001}


In [46]:
grid_result.best_params_

{'alpha': 0.001}

In [47]:
# 使用最佳參數重新建立模型
lasso_bestparam = linear_model.Lasso(alpha=grid_result.best_params_['alpha'])

# 訓練模型
lasso_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = lasso_bestparam.predict(x_test)

# 查看使用最佳參數模型的預測能力 (印出 MSE)
print(metrics.mean_squared_error(y_test, y_pred))

2845.997724429663


可以觀察到在選用最佳 α=0.001 的情況下，MSE 已下降 

### 延續 Day 40 作業，觀察使用 LASSO 於 boston 資料集上進行α調整的結果

In [34]:
# 載入 boston 資料集
boston = datasets.load_boston()
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.25, random_state=42)

# 建立 LASSO 線性回歸模型，與 Day 40 一樣使用α = 2.0
lasso = linear_model.Lasso(alpha=2.0)

# 查看在α=2.0，MSE 為何？
lasso.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

29.3974017773979


In [39]:
# 設定要訓練的超參數組合
alpha = [0.05, 0.1, 0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
param_grid = dict(alpha=alpha)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(lasso, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 8 種參數組合，總共要 train 24 次模型

Fitting 3 folds for each of 8 candidates, totalling 24 fits


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.0s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [40]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -24.840159 using {'alpha': 0.05}


In [41]:
grid_result.best_params_

{'alpha': 0.05}

In [42]:
# 使用最佳參數重新建立模型
lasso_bestparam = linear_model.Lasso(alpha=grid_result.best_params_['alpha'])

# 訓練模型
lasso_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = lasso_bestparam.predict(x_test)

# 查看使用最佳參數模型的預測能力 (印出 MSE)
print(metrics.mean_squared_error(y_test, y_pred))

23.38825744913904
